# Reproducing Figure 1 from "A Scale-Dependent Measure of System Dimensionality"

## Introduction

Complex systems, such as the brain neural system, are often characterized by a number of latent variables which is known as the dimensionality of a system. The correlation dimension and participation ratio quanitfy the dimensionality of such complex and dynamic systems locally and globally as can be observed by the executing the script below.

This script allows you to reproduce the results shown in figure 1 of the AccelNet paper on "A Scale-dependent Measure of System Dimensionality."

Please note that we have created a separate .py file consisting of the functions which are used throughout the code.

<div>
<img src="https://pbs.twimg.com/profile_images/1133961932950532096/15M5Fvdy_400x400.png" width="200" height="200"/>
</div>

<div>
<img src="https://cneuro-web01.s.uw.edu/wordpress/wp-content/uploads/2018/06/CNC-full-color-e1528133123627.png" width="242" height="83"/>
</div>

<div>
<img src="https://uw-s3-cdn.s3.us-west-2.amazonaws.com/wp-content/uploads/sites/98/2014/10/07214222/Signature_Center_Gold_RGB.png" width="242" height="83"/>
</div>

The following code segment imports the required packages in order to be able to run the code and reproduce the figures. Please note that the import accelnetfunctions line allows you to import and make use of critical functions to reproduce the images shown in figure 1 of the paper.

In [ ]:
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.metrics as sklm
import matplotlib as mpl
mpl.rc('image', cmap='Paired')
import scipy.stats as stats
from scipy.special import zeta as zeta
#import tensorflow as tf
from sklearn.decomposition import PCA
import random
import matplotlib.colors as colors
import matplotlib.cm as cmx
jet = cm = plt.get_cmap('Paired')
cNorm  = colors.Normalize(vmin=0, vmax=12)
cMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

from matplotlib.colors import ListedColormap


import accelnetfunctions

The following code segment enables you to graph the Lorenz attractor without considering the noise levels. Please note that the preprint uses 10,000 points which you may or may not be able to run the following code with depending on the tool you use (as documented below). The segment below performs analysis on the spiral.

In [ ]:
#N_points = 10000 #used in the preprint
#N_points = 10000 #tested on standard google colab
#N_points = 1000 #enough for a quick test
N_points = 5000 #works on syzygy.ca
x = accelnetfunctions.dataset_lorenz(N_points)
dists_matrix = sklm.pairwise_distances(x, n_jobs = -1)
dists = dists_matrix[np.triu_indices_from(dists_matrix, k=1)]
dists = dists[dists > 0]

The segment below produces a plot of the Lorenz attractor without noise.

In [ ]:
fig = plt.figure()
N_points = 1000
plt.scatter(x[:N_points,0], x[:N_points,1], s=3, c='black', marker='o', rasterized=True)
plt.savefig('SpiralExampleNoNoise.pdf')
plt.savefig('SpiralExampleNoNoise.svg')
plt.show()

The following code segment below performs analysis of the spiral with consideration of local noise levels.

In [ ]:
#N_points = 10000 #used in the preprint
#N_points = 10000 #tested on standard google colab
#N_points = 1000 #enough for a quick test
N_points = 5000 #works on syzygy.ca
x = accelnetfunctions.dataset_spiral(N_points, .1)
dists_matrix = sklm.pairwise_distances(x, n_jobs = -1)
dists = dists_matrix[np.triu_indices_from(dists_matrix, k=1)]
dists = dists[dists > 0]

The following code segment allows you to plot the spiral shape of the Lorenz attractor with consideration of local noise levels.

In [ ]:
fig = plt.figure()
N_points = 500000
plt.scatter(x[:N_points,0], x[:N_points,1], s=3, c='black', marker='o', rasterized=True)
plt.savefig('NoisySpiralExample.pdf')
plt.savefig('NoisySpiralExample.svg')
plt.show()

The following code segment performs a correlation dimension analysis. The shape of the graph shows how the correlation dimension approaches 0 on a larger scale. The correlation dimension is determined from scaling the number of pair of data points with a separation of less than r as r approaches 0. The correlation dimension is accurate to use for small scales, but it approaches a value of 0 on larger scales.

In [ ]:
scales = accelnetfunctions.scalelog(dists, dists=True, Npoints=100)
Dcorr, logmidpoints = accelnetfunctions.boot_dim_corr(dists_matrix, scales, boot_pcg=0.8, N_bootstrap=2)
Dcorrmean = np.nanmean(Dcorr, axis = 0)
Dcorrstd = np.nanstd(Dcorr, axis = 0)
plt.plot(logmidpoints[:], Dcorrmean[:])
# plt.scatter(logmidpoints[:], Dcorrmean[:])
plt.fill_between(logmidpoints, Dcorrmean - Dcorrstd, Dcorrmean + Dcorrstd, color='gray', alpha=0.2)
plt.xlabel('log scale')
plt.ylabel('corr dim')
# plt.xlim([-5,1])
plt.ylim([0,3.])
plt.savefig('SpiralExampleCorrDim.pdf')
plt.savefig('SpiralExampleCorrDim.svg')
plt.show()

The following code segment performs a participation ratio analysis. The shape of the graph shows that the participation ratio remains an accurate representation of the dimensionality of the system as we look at larger scales. The participation ratio counts the effective dimensions of the spread of data by taking the ratio of the square of the first and second moments of the eigenvalue probability density function. The number of effective dimensions interpolates to the participation ratio globally; it intuitively makes sense in dynamic systems such as the Lorenz attractor (which is what we perform our analysis on in our execution of the code segment below). Therefore, we can use this value to increase our understanding of the dimensionality of complex neural systems.

In [ ]:
PRs, Scales = accelnetfunctions.boot_dim_pr(x, dists_matrix, scales, N_points=1000, N_maxiters=10000, C_minsize=50)
PRmean =  np.mean(PRs, axis=1)
plt.plot(np.log(scales), PRmean)
# iql = scipy.stats.iqr(PRs, axis=1, rng=(34,50), nan_policy='omit')
# iqu = scipy.stats.iqr(PRs, axis=1, rng=(50,66), nan_policy='omit')
# plt.fill_between(np.log(sigma_space), PRmean - iql, PRmean + iqu, color='gray', alpha=0.2)
PRste = np.nanstd(PRs, axis=1) #/ np.sqrt(N_points)
plt.fill_between(np.log(scales), PRmean - PRste, PRmean + PRste, color='gray', alpha=0.2)
plt.xlabel('log scale')
plt.ylabel('PR dim')
plt.ylim([0,3.])
aplt.savefig('SpiralExamplePR.pdf')
plt.show()